# Ejemplo de personalización de usuarios de AWS de Amazon Personalize + recomendaciones contextuales


## Introducción <a class="anchor" id="intro"></a>

Por lo general, los algoritmos en Amazon Personalize (llamados recetas) buscan resolver diferentes tareas que se explican a continuación:

1. **User Personalization (Personalización del usuario)**: recomienda elementos en base a las interacciones previas del usuario con los elementos.
1. **Personalized-Ranking**: toma una recopilación de elementos y, luego, los ubica en un posible orden de interés mediante un enfoque similar al de una HRNN.
1. **SIMS (elementos similares)**: en base a un elemento, recomienda otros elementos con los que también interactúan los usuarios.

Sin importar cuál sea el caso de uso, todos los algoritmos comparten una base de aprendizaje sobre los datos de interacción entre el usuario y los elementos, la cual se define según 3 atributos principales:

1. **UserID**: el usuario que interactuó
1. **ItemID**: el elemento con el que interactuó el usuario
1. **Timestamp (Marca temporal)**: la hora a la que ocurrió la interacción


## Elija un conjunto de datos o un origen de datos <a class="anchor" id="source"></a>
[Regresar al principio](#top)

Como ya lo mencionamos, los datos de la interacción del usuario con los elementos son cruciales para comenzar con el servicio. Esto significa que necesitamos buscar los casos de uso que generan ese tipo de datos, algunos ejemplos comunes son los siguientes:

1. Aplicaciones de video bajo demanda
1. Plataformas de comercio electrónico
1. Agregadores/plataformas de redes sociales

Existen algunas pautas para evaluar un problema que sea adecuado para Personalize. Como punto inicial, recomendamos los valores que figuran a continuación, aunque los [límites oficiales](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) son un poco más bajos.

* Usuarios autenticados
* Al menos 50 usuarios exclusivos
* Al menos 100 elementos exclusivos
* Al menos 2 docenas de interacciones para cada usuario 

La mayoría de las veces esto es fácil de lograr, y si no llega a esta cantidad en alguna categoría, puede compensarlo con una mayor cantidad en otra categoría.

En general, Personalize no enviará los datos en forma perfecta y deberá realizar algunas modificaciones para que la estructura sea la correcta. Este cuaderno tiene como fin guiarle en este proceso. 

Para comenzar, utilizaremos un conjunto de datos de revisión de aerolíneas. Una recopilación del conjunto de datos creado a partir de todas las opiniones de los usuarios que se encuentran en Skytrax (www.airlinequality.com). Los datos pueden encontrarse en https://github.com/quankiquanki/skytrax-reviews-dataset 

In [274]:
import pandas as pd, numpy as np
import io
import scipy.sparse as ss
import json
import time
import datetime
import os
import sagemaker.amazon.common as smac
import boto3
import uuid
from botocore.exceptions import ClientError

### Importar y explorar el conjunto de datos

In [275]:
data_dir = "airlines_data"
!mkdir $data_dir
!cd $data_dir && wget https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv


--2020-06-18 16:33:44--  https://raw.githubusercontent.com/quankiquanki/skytrax-reviews-dataset/master/data/airline.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.64.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.64.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34752262 (33M) [text/plain]
Saving to: ‘airline.csv’

airline.csv         100%[===================>]  33.14M   113MB/s    in 0.3s    

2020-06-18 16:33:45 (113 MB/s) - ‘airline.csv’ saved [34752262/34752262]



In [276]:
airline_df = pd.read_csv(data_dir + '/airline.csv')
airline_df.head()

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,route,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,NaN,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1
1,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Ron Kuhlmann,United States,2015-01-05,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,NaN,NaN,Business Class,NaN,10.0,4.0,5.0,4.0,1.0,NaN,NaN,5.0,1
2,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,E Albin,Switzerland,2014-09-14,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,NaN,NaN,Economy,NaN,9.0,5.0,5.0,4.0,0.0,NaN,NaN,5.0,1
3,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,Tercon Bojan,Singapore,2014-09-06,Adria serves this 100 min flight from Ljubljan...,NaN,NaN,Business Class,NaN,8.0,4.0,4.0,3.0,1.0,NaN,NaN,4.0,1
4,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,L James,Poland,2014-06-16,WAW-SKJ Economy. No free snacks or drinks on t...,NaN,NaN,Economy,NaN,4.0,4.0,2.0,1.0,2.0,NaN,NaN,2.0,0


Como podemos ver, el conjunto de datos tiene muchas columnas que podemos utilizar para crear los conjuntos de datos necesarios en Amazon Personalize.

Lo primero que haremos es hacer 2 copias del conjunto de datos.

In [277]:
a_interactions_df = airline_df.copy()
a_users_df = airline_df.copy()

## Creación del conjunto de datos de las interacciones

Crearemos el conjunto de datos de las interacciones. Para ello, seguiremos estos pasos:

- Eliminar las columnas que no nos interesan
- Crear una columna nueva para incluir el tipo de evento
- Cambiar el nombre de las columnas por una convención de nomenclatura más estándar para su trabajo de importación de Amazon Personalize



In [278]:
# Keeping only 5 columns
a_interactions_df = a_interactions_df[['airline_name', 'author', 'date', 'cabin_flown', 'overall_rating']]
# Creating an additional column for Event Type
a_interactions_df['EVENT_TYPE']='RATING'
# Making sure the author name is unique without spaces
a_interactions_df['author'] = a_interactions_df['author'].str.replace(" ","")
# Rename the columns to a more Amazon Personalize standar notation
a_interactions_df.rename(columns = {'airline_name':'ITEM_ID', 'author':'USER_ID',
                              'date':'TIMESTAMP', 'cabin_flown': 'CABIN_TYPE', 'overall_rating': 'EVENT_VALUE'}, inplace = True) 
a_interactions_df.head()

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,2015-04-10,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,2015-01-05,Business Class,10.0,RATING
2,adria-airways,EAlbin,2014-09-14,Economy,9.0,RATING
3,adria-airways,TerconBojan,2014-09-06,Business Class,8.0,RATING
4,adria-airways,LJames,2014-06-16,Economy,4.0,RATING


Amazon Personalize es compatible con las **recomendaciones contextuales**, a través de las cuales se puede mejorar la relevancia de las recomendaciones gracias a generarlas dentro de un contexto, por ejemplo, tipo de dispositivo, ubicación, hora del día, etc. La información contextual también es útil en la personalización de usuarios nuevos o no identificados, incluso cuando no se conocen las interacciones anteriores de estos usuarios.

En nuestro caso utilizaremos el **Cabin Type (tipo de cabina)** como contexto para recomendar qué aerolínea es la más adecuada para nuestro usuario. Analicemos qué valores podremos incluir como nuestro contexto a la hora de obtener recomendaciones.


In [279]:
a_interactions_df.CABIN_TYPE.unique()

array(['Economy', 'Business Class', nan, 'Premium Economy', 'First Class'],
      dtype=object)

Como podemos ver, nuestro valor de **Timestamp (Marca temporal)** actual en el conjunto de datos es una cadena. Amazon Personalize requiere que el valor de marca temporal sea de tipo Unix. Tomemos un valor de marca temporal aleatorio y convirtámoslo a tipo Unix.

In [280]:
# Get a random value from the timestamp column
arb_time_stamp = a_interactions_df.iloc[50]['TIMESTAMP']
# Transform this string to date time
date_time_obj = datetime.datetime.strptime(arb_time_stamp, '%Y-%m-%d')
print('Date:', date_time_obj.date())
# Get the date of this object
d = date_time_obj.date()
# Transform the date object to Unix time
unixtime = time.mktime(d.timetuple())
print('Unix Time: ', unixtime)

Date: 2004-03-18
Unix Time:  1079568000.0


Ahora haremos la misma transformación en todos nuestros valores en la columna de marca temporal.

In [281]:
# Define a function
def convert_to_unix(string_date):
    date_time_obj = datetime.datetime.strptime(string_date, '%Y-%m-%d')
    d = date_time_obj.date()
    return time.mktime(d.timetuple())

# Apply this function across the Timestamp column
a_interactions_df['TIMESTAMP'] = a_interactions_df['TIMESTAMP'].apply(convert_to_unix)
a_interactions_df.head(5)

,ITEM_ID,USER_ID,TIMESTAMP,CABIN_TYPE,EVENT_VALUE,EVENT_TYPE
0,adria-airways,DIto,1.428624e+09,Economy,7.0,RATING
1,adria-airways,RonKuhlmann,1.420416e+09,Business Class,10.0,RATING
2,adria-airways,EAlbin,1.410653e+09,Economy,9.0,RATING
3,adria-airways,TerconBojan,1.409962e+09,Business Class,8.0,RATING
4,adria-airways,LJames,1.402877e+09,Economy,4.0,RATING


Echemos un vistazo a algunas de las propiedades de nuestro conjunto de datos.

In [282]:
a_interactions_df.describe()

,TIMESTAMP,EVENT_VALUE
count,4.139600e+04,36861.000000
mean,1.373950e+09,6.039527
std,5.771909e+07,3.214680
min,0.000000e+00,1.000000
25%,1.350864e+09,3.000000
50%,1.389658e+09,7.000000
75%,1.412122e+09,9.000000
max,1.438474e+09,10.000000


¿Hay algún valor nulo?

In [283]:
a_interactions_df.isnull().values.any()

True

Eliminemos esos valores nulos y asegurémonos de que no haya ninguno.

In [284]:
a_interactions_df = a_interactions_df.dropna()
a_interactions_df.isnull().values.any()

False

Ahora que tenemos nuestros datos listos para Amazon Personalize, guardémoslos en un archivo local.

In [285]:
interactions_filename = "a_interactions.csv"
a_interactions_df.to_csv((data_dir + "/"+interactions_filename), index=False, float_format='%.0f')

## Creación del conjunto de datos para los usuarios

Creemos el conjunto de datos para los usuarios. Para ello, seguiremos estos pasos:

- Eliminar las columnas que no nos interesan
- Crear una columna nueva para incluir la nacionalidad como metadatos del usuario
- Cambiar el nombre de las columnas por una convención de nomenclatura más estándar para su trabajo de importación de Amazon Personalize


In [286]:
# Copy the complete airlines data set
a_users_df = airline_df.copy()
# Select only interested columns
a_users_df = a_users_df[['author', 'author_country']]
# Clean up the authors string
a_users_df['author'] = a_users_df['author'].str.replace(" ","")
# Rename the columns
a_users_df.rename(columns = { 'author':'USER_ID', 'author_country':'NATIONALITY'}, inplace = True) 
# Drop any null values
a_users_df = a_users_df.dropna()
# Save your file locally
users_filename = "a_users.csv"
a_users_df.to_csv((data_dir +"/"+users_filename), index=False)

## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>
[Regresar al principio](#top)

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

In [288]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Los nombres de los buckets de Amazon S3 son exclusivos a nivel global. Para crear un nombre de bucket exclusivo, el siguiente código agregará la cadena `personalizepoc` a su número de cuenta de AWS. Luego, crea un bucket con este nombre en la región descubierta en la celda anterior.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-user-personalization-example" + suffix
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos. 

In [294]:
interactions_filename = data_dir + '/a_interactions.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_filename)

In [295]:
user_metadata_file = data_dir + '/a_users.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_metadata_file).upload_file(user_metadata_file)

## Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones <a class="anchor" id="group_dataset"></a>
[Regresar al principio](#top)

El nivel más alto de aislamiento y abstracción con Amazon Personalize es un *grupo de conjuntos de datos*. La información almacenada dentro de uno de estos grupos de conjunto de datos no afecta ninguno de los demás grupos de conjuntos de datos o modelos creados de uno de ellos; están completamente aislados. Esto permite ejecutar muchos experimentos y es parte de lo que hacemos para mantener a los modelos de forma privada y completamente entrenados en sus datos. 

Antes de importar los datos que preparó antes, debe haber un grupo de conjuntos de datos y un conjunto de datos agregados a este que maneje las interacciones.

Los grupos de conjuntos de datos pueden almacenar los siguientes tipos de información:

* Interacciones entre el usuario y el elemento
* Flujo de eventos (interacciones en tiempo real)
* Metadatos del usuario
* Metadatos del elemento

Antes de crear el grupo de conjuntos de datos y el conjunto de datos para nuestros datos de interacción, validemos que su entorno se pueda comunicar exitosamente con Amazon Personalize.

In [293]:
personalize = boto3.client(service_name='personalize')
personalize_runtime = boto3.client(service_name='personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

### Crear un grupo de conjuntos de datos

La siguiente celda creará un nuevo grupo de conjuntos de datos con el nombre *airlines-dataset-group* + un sufijo.

In [64]:
dataset_group_name = "airlines-dataset-group-" + suffix

create_dataset_group_response = personalize.create_dataset_group(
    name = dataset_group_name
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/airlines-dataset-group-55035",
  "ResponseMetadata": {
    "RequestId": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:14:12 GMT",
      "x-amzn-requestid": "a8bb75fb-f15b-45da-997e-08eb14d7733a",
      "content-length": "107",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, este debe estar activo. Esto puede llevar uno o dos minutos. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del grupo de conjuntos de datos a cada segundo, hasta por un máximo de 3 horas.

In [65]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(20)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Ahora que tiene un grupo de conjuntos de datos, puede crear un conjunto de datos para los datos de interacción.

# Crear conjuntos de datos

### Conjunto de datos de interacciones

Primero, defina un esquema para indicar a Amazon Personalize el tipo de conjunto de datos que está cargando. Existen varias palabras clave reservadas y obligatorias, necesarias para el esquema, según el tipo de conjunto de datos. Para obtener más información, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Aquí, creará un esquema para datos de interacciones, para lo que se necesitan los campos `USER_ID`, `ITEM_ID`, `TIMESTAMP`, `CABIN_TYPE`, `EVENT_TYPE`, `EVENT_VALUE` y `TIMESTAMP`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.

In [55]:
schema_name="airlines-interaction-schema-"+suffix

In [56]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name":"CABIN_TYPE",
            "type": "string",
            "categorical": True
        },
        {
          "name": "EVENT_TYPE",
          "type": "string"
        },
        {
          "name": "EVENT_VALUE",
          "type": "float"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = schema_name,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-interaction-schema-55035",
  "ResponseMetadata": {
    "RequestId": "4e045a61-d479-485c-93ff-6072076ccaa9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:10:34 GMT",
      "x-amzn-requestid": "4e045a61-d479-485c-93ff-6072076ccaa9",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [66]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn,
    name = "airlines-dataset-interactions-" + suffix
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/airlines-dataset-group-55035/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "968e6cac-310a-4889-8243-e86ef90696ed",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:15:14 GMT",
      "x-amzn-requestid": "968e6cac-310a-4889-8243-e86ef90696ed",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Conjuntos de datos de los usuarios

Aquí, creará un esquema para los datos de metadatos de elementos, para lo que se necesitan los campos `USER_ID` y `NATIONALITY`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.


In [62]:
metadata_schema_name="airlines-users-schema-"+suffix

In [63]:
metadata_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "NATIONALITY",
            "type": "string",
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = metadata_schema_name,
    schema = json.dumps(metadata_schema)
)

metadata_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))


{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/airlines-users-schema-55035",
  "ResponseMetadata": {
    "RequestId": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:13:50 GMT",
      "x-amzn-requestid": "17844e2f-860d-484a-bb39-ab4a10e7b9fd",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
dataset_type = "USERS"
create_metadata_dataset_response = personalize.create_dataset(
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = metadata_schema_arn,
    name = "airlines-metadata-dataset-users-" + suffix
)

metadata_dataset_arn = create_metadata_dataset_response['datasetArn']
print(json.dumps(create_metadata_dataset_response, indent=2))

## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

### Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

In [69]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy));

### Crear un rol de IAM

Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Creemos un rol de IAM y adjuntemos las políticas necesarias a este. El siguiente código adjunta políticas muy permisivas. Utilice políticas más restrictivas para cualquier aplicación en producción.

In [70]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role-"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}
try:
    create_role_response = iam.create_role(
        RoleName = role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
    );

    iam.attach_role_policy(
        RoleName = role_name,
        PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
    );

    role_arn = create_role_response["Role"]["Arn"]
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise
        
# sometimes need to wait a bit for the role to be created
time.sleep(45)
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeS3Role-55035


# Crear trabajos de importación de conjuntos de datos

## Importar los datos de las interacciones <a class="anchor" id="import"></a>

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [109]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-dataset-import-job-"+suffix,
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "d118cf11-b568-4767-99d5-30a15871981a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:38 GMT",
      "x-amzn-requestid": "d118cf11-b568-4767-99d5-30a15871981a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Importe los datos de los usuarios <a class="anchor" id="import"></a>

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [110]:
create_metadata_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "airlines-users-metadata-dataset-import-job-"+suffix,
    datasetArn = metadata_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_metadata_file)
    },
    roleArn = role_arn
)

metadata_dataset_import_job_arn = create_metadata_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_metadata_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/airlines-users-metadata-dataset-import-job-14078",
  "ResponseMetadata": {
    "RequestId": "679d9401-568d-45e0-ba8c-df8f1574228d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 21:46:40 GMT",
      "x-amzn-requestid": "679d9401-568d-45e0-ba8c-df8f1574228d",
      "content-length": "136",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Esperar a que el trabajo de importación de conjuntos de datos tenga un estado ACTIVO

Antes de que podamos utilizar el grupo de conjuntos de datos, el trabajo de importación debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del trabajo de importación a cada segundo, por hasta un máximo de 3 horas.

La importación de los datos puede llevar algo de tiempo, según el tamaño del conjunto de datos. En esta demostración, el trabajo de importación de datos se ha ejecutado antes.

In [111]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


In [112]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = metadata_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


Cuando la importación del conjunto de datos esté activa, estará listo para empezar a crear modelos con la receta de personalización de usuarios de AWS.

## Crear soluciones <a class="anchor" id="solutions"></a>
[Regresar al principio](#top)

En este cuaderno, crearemos soluciones con la siguiente receta:

1. aws-user-personalization


Una variación específica de un algoritmo se denomina receta en Amazon Personalize. Las diferentes recetas son adecuadas para diferentes situaciones. Un modelo entrenado se denomina solución, y cada solución puede tener muchas versiones relacionadas con un determinado volumen de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que son compatibles. Esto le permitirá seleccionar una y utilizarla para crear el modelo.

In [113]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


El resultado es solo una representación JSON de todos los algoritmos mencionados en la introducción.

A continuación, seleccionaremos recetas específicas y crearemos modelos con ellas.

### Personalización del usuario de AWS

La personalización de usuario de AWS es uno de los modelos de recomendación más avanzados que puede utilizar y que permite actualizaciones en tiempo real de recomendaciones basadas en el comportamiento del usuario. También tiende a conseguir mejores resultados que otros enfoques, como el filtrado colaborativo. La receta requiere el entrenamiento más largo, así que comencemos con esta receta primero.

Para nuestro caso de uso, mediante los datos de revisión de aerolíneas, podemos utilizar el algoritmo de personalización del usuario de AWS para recomendar nuevas aerolíneas a un usuario en base al comportamiento previo de etiquetado de artistas de un usuario. Recuerde que hemos utilizado datos de etiquetas para representar interacciones positivas entre un usuario y un artista.

En primer lugar, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [114]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Crear la solución

Primero cree una solución con la receta. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

Tenga en cuenta que aquí está activada la HPO. Esto es una buena idea en las siguientes ocasiones. 


In [ ]:
create_solution_response = personalize.create_solution(
    name = "airlines-user-personalization-solution-HPO-"+suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 40 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [117]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "ResponseMetadata": {
    "RequestId": "148a0fbd-5465-4619-ac90-449c0ef23b73",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 22:01:32 GMT",
      "x-amzn-requestid": "148a0fbd-5465-4619-ac90-449c0ef23b73",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [118]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

## Evaluar las versiones de las soluciones<a class="anchor" id="eval"></a>
[Regresar al principio](#top)

No debería tardar más de una hora en entrenar todas las soluciones de este cuaderno. Mientras se realiza el entrenamiento, se recomienda dedicar tiempo a leer en detalle los distintos algoritmos (recetas) y su comportamiento. Este es también un buen momento para considerar alternativas a la forma en que los datos se introdujeron en el sistema y el tipo de resultados que espera ver.

Cuando las soluciones terminan de crearse, el siguiente paso es obtener las métricas de evaluación. Personalize calcula estas métricas basándose en un subconjunto de los datos de entrenamiento. La siguiente imagen muestra cómo Personalize divide los datos. Dados 10 usuarios, con 10 interacciones cada uno (un círculo representa una interacción), las interacciones se ordenan de la más antigua a la más reciente en función de la marca temporal. Personalize utiliza todos los datos de interacción del 90 % de los usuarios (círculos azules) para entrenar la versión de la solución, y el 10 % restante para la evaluación. Para cada uno de los usuarios del 10 % restante, el 90 % de sus datos de interacción (círculos verdes) se utiliza como entrada para la llamada al modelo entrenado. El 10 % restante de sus datos (círculo naranja) se compara con la salida producida por el modelo y se utiliza para calcular las métricas de evaluación.

![métricas de personalize](static/imgs/personalize_metrics.png)

Recomendamos leer [la documentación](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) para comprender las métricas, pero, además, copiamos partes de la documentación a continuación para mayor comodidad.

Es necesario que comprenda los siguientes términos relativos a la evaluación en Personalize:

* *Recomendación relevante* se refiere a una recomendación que coincide con un valor en los datos de prueba para el usuario en particular.
* *Rango* se refiere a la posición de un elemento recomendado en la lista de recomendaciones. Se supone que la posición 1 (la primera de la lista) es la más relevante para el usuario.
* *Consulta* se refiere al equivalente interno de una llamada a GetRecommendations.

Las métricas producidas por Personalize son las siguientes:
- **coverage**: La proporción de elementos únicos recomendados de todas las consultas sobre el número total de elementos únicos en los datos de entrenamiento (incluye tanto los conjuntos de datos de elementos como de interacciones).
- **mean_reciprocal_rank_at_25**: La [media de los rangos recíprocos](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la primera recomendación relevante entre las 25 primeras recomendaciones sobre todas las consultas. Esta métrica es apropiada si está interesado en la recomendación de mayor rango.
- **normalized_discounted_cumulative_gain_at_K**: La ganancia descontada supone que las recomendaciones más bajas de una lista de recomendaciones son menos relevantes que las más altas. Por lo tanto, cada recomendación se descuenta (se le da un peso menor) por un factor que depende de su posición. A fin de obtener la [ganancia descontada acumulada](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) en K, se suma cada recomendación descontada relevante en las  recomendaciones más importantes de K. La ganancia acumulada descontada normalizada (NDCG) es la DCG dividida por la DCG ideal, de manera que la NDCG está entre 0 y 1. (El DCG ideal es aquel en el que las recomendaciones más importantes de K están ordenadas por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/log (1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica recompensa los elementos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.
- **precision_at_K**: El número de recomendaciones relevantes de las principales recomendaciones de K dividido por K. Esta métrica recompensa la recomendación precisa de los elementos relevantes.

Veamos las métricas de evaluación de cada una de las soluciones elaboradas en este cuaderno. *Tenga en cuenta que sus resultados pueden diferir de los descritos en este cuaderno debido a la calidad del conjunto de datos de LastFM.* 

### Métricas de personalización del usuario de AWS

En primer lugar, recupere las métricas de evaluación para la versión de la solución de personalización de los usuarios de AWS.

In [120]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))


{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/airlines-hrnn-metadata-solution-HPO-14078/54a6c563",
  "metrics": {
    "coverage": 0.4046,
    "mean_reciprocal_rank_at_25": 0.2035,
    "normalized_discounted_cumulative_gain_at_10": 0.2909,
    "normalized_discounted_cumulative_gain_at_25": 0.3174,
    "normalized_discounted_cumulative_gain_at_5": 0.2418,
    "precision_at_10": 0.0444,
    "precision_at_25": 0.022,
    "precision_at_5": 0.0605
  },
  "ResponseMetadata": {
    "RequestId": "156a6e70-152b-4940-8b17-048252419fd0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:02:24 GMT",
      "x-amzn-requestid": "156a6e70-152b-4940-8b17-048252419fd0",
      "content-length": "424",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


# Crear una campaña para la solución

## Crear una campaña <a class="anchor" id="create"></a>

Una campaña es una versión de la solución alojada; un punto de conexión que puede consultar para obtener recomendaciones. El precio se fija estimando la capacidad de rendimiento (solicitudes de personalización de los usuarios por segundo). Cuando implementa una campaña, se establece un valor mínimo de rendimiento por segundo (TPS). Este servicio, como muchos dentro de AWS, se escalará automáticamente en función de la demanda, pero si la latencia es crítica, es posible que desee aprovisionar por adelantado para una mayor demanda. Para esta POC y la demostración, todos los límites mínimos de rendimiento se establecen en 1. Para obtener más información, consulte la [página de precios](https://aws.amazon.com/personalize/pricing/).

Empecemos a implementar las campañas.

### Personalización del usuario de AWS

Implemente una campaña para su versión de la solución de personalización de los usuarios de AWS. La implementación de una campaña puede tardar unos 10 minutos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas y el objetivo aquí es crear múltiples campañas. Así que estableceremos el bucle while para todas las campañas más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [122]:
create_campaign_response = personalize.create_campaign(
    name = "airlines-metadata-campaign-"+suffix,
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 2,    
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/airlines-metadata-campaign-14078",
  "ResponseMetadata": {
    "RequestId": "4c882630-86aa-4c5d-accd-75225f9804a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Mon, 15 Jun 2020 23:25:37 GMT",
      "x-amzn-requestid": "4c882630-86aa-4c5d-accd-75225f9804a4",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [123]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


### Personalización del usuario de AWS

La personalización del usuario de AWS es uno de los algoritmos más avanzados que ofrece Amazon Personalize. Admite la personalización de los elementos para un usuario concreto en función de su comportamiento anterior y puede captar eventos en tiempo real a fin de modificar las recomendaciones para un usuario sin necesidad de volver a entrenarlo. 

Como el algoritmo de personalización de los usuarios de AWS se basa en tener una muestra de usuarios, carguemos los datos que necesitamos para ello y seleccionemos tres usuarios al azar.

In [262]:
users_df = pd.read_csv(data_dir + '/a_users.csv')
# Render some sample data
users_df.sample(5)

,USER_ID,NATIONALITY
3918,JHartley,United Kingdom
27477,DDriscoll,United Kingdom
19563,BrianElliott,United Kingdom
22989,AHornbuckle,Australia
35724,CMoon,United Kingdom


Ahora, presentamos las recomendaciones para los 3 usuarios seleccionados al azar. A continuación, exploraremos las interacciones en tiempo real antes de pasar a la clasificación personalizada.

Una vez más, creamos una función de ayuda para mostrar los resultados en un marco de datos agradable.

#### Resultados de las llamadas a la API

In [165]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    
#   Context Recommendations
    context_options = ['None','Economy', 'Business Class','Premium Economy', 'First Class']
    
    for context in context_options:
        # Get the recommendations
        if context=='none':
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
            )
        else:
            get_recommendations_response = personalize_runtime.get_recommendations(
                campaignArn = campaign_arn,
                userId = str(user_id),
                context = {
                  'CABIN_TYPE': context
                }
            )
        # Build a new dataframe of recommendations
        item_list = get_recommendations_response['itemList']
        recommendation_list = []
        for item in item_list:
            recommendation_list.append(item['itemId'])
    #     print(recommendation_list)
        new_rec_DF = pd.DataFrame(recommendation_list, columns = [context])
        # Add this dataframe to the old one
        recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [263]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID     NATIONALITY
37013    RDow  United Kingdom


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,thomson-airways,british-airways,thomson-airways,united-airlines
1,united-airlines,thomas-cook-airlines,virgin-atlantic-airways,virgin-atlantic-airways,british-airways
2,thomson-airways,united-airlines,turkish-airlines,air-new-zealand,american-airlines
3,virgin-atlantic-airways,easyjet,united-airlines,british-airways,china-southern-airlines
4,air-new-zealand,monarch-airlines,china-southern-airlines,united-airlines,delta-air-lines
5,turkish-airlines,virgin-atlantic-airways,qatar-airways,thomas-cook-airlines,lufthansa
6,china-southern-airlines,british-airways,emirates,turkish-airlines,alaska-airlines
7,thomas-cook-airlines,jet2-com,air-france,monarch-airlines,virgin-atlantic-airways
8,lufthansa,lufthansa,american-airlines,china-southern-airlines,us-airways
9,american-airlines,american-airlines,lufthansa,eva-air,thomson-airways


In [264]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

      USER_ID NATIONALITY
26198   CJeff   Singapore


,None,Economy,Business Class,Premium Economy,First Class
0,philippine-airlines,philippine-airlines,cathay-pacific-airways,cathay-pacific-airways,singapore-airlines
1,cathay-pacific-airways,singapore-airlines,philippine-airlines,air-france,thai-airways
2,singapore-airlines,tigerair,malaysia-airlines,eva-air,philippine-airlines
3,ana-all-nippon-airways,jetstar-asia,srilankan-airlines,air-new-zealand,delta-air-lines
4,thai-airways,cathay-pacific-airways,thai-airways,philippine-airlines,emirates
5,air-india,airasia,singapore-airlines,klm-royal-dutch-airlines,ana-all-nippon-airways
6,china-eastern-airlines,cebu-pacific,air-india,airasia-x,china-southern-airlines
7,dragonair,scoot,emirates,qantas-airways,alaska-airlines
8,srilankan-airlines,ana-all-nippon-airways,ana-all-nippon-airways,china-southern-airlines,american-airlines
9,air-france,dragonair,asiana-airlines,united-airlines,china-eastern-airlines


Aquí vemos claramente que las recomendaciones para cada usuario son diferentes. Si necesitara un caché para estos resultados, podría empezar por ejecutar las llamadas a la API a través de todos sus usuarios y almacenar los resultados, o bien podría utilizar una exportación por lotes, que se tratará más adelante en este cuaderno.

El siguiente tema es el de los eventos en tiempo real. Personalize tiene la capacidad de escuchar los eventos de su aplicación para actualizar las recomendaciones mostradas al usuario. Esto es especialmente útil en las cargas de trabajo de los medios de comunicación, como el video bajo demanda, donde la intención de un cliente puede variar en función de si lo está viendo con sus hijos o solo.

Además, los eventos que se registran a través de este sistema se almacenan hasta que se produce una llamada de eliminación por su parte y se utilizan como datos históricos junto con los demás datos de interacción que ha proporcionado cuando entrena sus próximos modelos.

#### Eventos en tiempo real

Comience por crear un rastreador de eventos que se adjunte a la campaña.

In [150]:
response = personalize.create_event_tracker(
    name='AirlinesEventsTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:144386903708:event-tracker/d2e7ccdc
820029aa-b00c-4eff-9e6f-60830bb68508


Crearemos un código que simule la interacción de un usuario con un elemento determinado. Después de ejecutar este código, obtendrá recomendaciones que se diferencian de los resultados anteriores.

Comenzamos creando algunos métodos para la simulación de eventos en tiempo real.

In [200]:
session_dict = {}

def send_user_rating(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
        "itemId": str(ITEM_ID),
        "eventValue": 10,
        "cabinType": "Economy"
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
        trackingId = TRACKING_ID,
        userId= str(USER_ID),
        sessionId = session_ID,
        eventList = [{
            'sentAt': int(time.time()),
            'eventType': 'RATING',
            'properties': event_json
            }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id):
    # Interact with the airline
    # Sending a rating of 10 in Economy class for the airline with that user
    send_user_rating(USER_ID=user_id, ITEM_ID=item_id)
    
    
    #   Context Recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
        context = {
          'CABIN_TYPE': 'Economy'
        }
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        recommendation_list.append(item['itemId'])
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [item_id+'|Economy'])
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df


Hasta ahora no hemos generado ningún evento en tiempo real, solo hemos configurado el código. Con el fin de comparar las recomendaciones antes y después de los eventos en tiempo real, seleccionemos un usuario y generemos las recomendaciones originales para él.

## Recomendaciones antes de utilizar el rastreador de eventos

In [265]:
recommendations_df_users = pd.DataFrame()
users = users_df.sample()
print(users)
users= users['USER_ID'].tolist()
for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)
user_id = users[0]
recommendations_df_users

        USER_ID NATIONALITY
6313  ACrociani       Italy


,None,Economy,Business Class,Premium Economy,First Class
0,british-airways,alitalia,iberia,british-airways,american-airlines
1,alitalia,brussels-airlines,british-airways,virgin-atlantic-airways,british-airways
2,iberia,ryanair,qatar-airways,united-airlines,delta-air-lines
3,brussels-airlines,easyjet,alitalia,brussels-airlines,united-airlines
4,qatar-airways,iberia,brussels-airlines,turkish-airlines,lufthansa
5,lufthansa,aer-lingus,emirates,alitalia,emirates
6,american-airlines,aegean-airlines,lufthansa,air-france,qatar-airways
7,united-airlines,lufthansa,turkish-airlines,lufthansa,swiss-international-air-lines
8,icelandair,qatar-airways,swiss-international-air-lines,icelandair,us-airways
9,delta-air-lines,tap-portugal,oman-air,delta-air-lines,iberia


In [266]:
user_id

'ACrociani'

Ahora tenemos una lista de recomendaciones para este usuario antes de aplicar cualquier evento en tiempo real. Ahora elegiremos 3 artistas al azar con los que simularemos que nuestro usuario interactúa y veremos cómo cambian las recomendaciones.

In [267]:
# Next generate 3 random Airlines
airlines = a_interactions_df.sample(3)['ITEM_ID'].tolist()
airlines

['thai-airways', 'austrian-airlines', 'united-airlines']

In [268]:
user_recommendations_df = pd.DataFrame()
# Note this will take about 15 seconds to complete due to the sleeps
for airline in airlines:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, airline)
    time.sleep(5)
print(user_id)
user_recommendations_df

ACrociani


,thai-airways|Economy,austrian-airlines|Economy,united-airlines|Economy
0,alitalia,ryanair,brussels-airlines
1,brussels-airlines,brussels-airlines,lufthansa
2,ryanair,easyjet,turkish-airlines
3,easyjet,tap-portugal,austrian-airlines
4,iberia,aegean-airlines,tap-portugal
5,aer-lingus,turkish-airlines,alitalia
6,aegean-airlines,lufthansa,germanwings
7,lufthansa,alitalia,aegean-airlines
8,qatar-airways,iberia,swiss-international-air-lines
9,tap-portugal,air-india,air-berlin


En la celda anterior, la primera columna después del índice muestra las recomendaciones por defecto del usuario desde el modelo de personalización de usuarios de AWS y cada columna posterior tiene un encabezado de las aerolíneas con las que interactuó a través de un evento en tiempo real y las recomendaciones después de que este evento ocurriera. 

Es posible que el comportamiento no cambie mucho. Esto se debe a la naturaleza relativamente limitada de este conjunto de datos. Si desea comprender mejor esto, intente simular la calificación de aerolíneas al azar con calificaciones aleatorias. Debería ver un impacto más pronunciado.